## BERT Model

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [3]:
train = pd.read_csv('CS 583 Project/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('CS 583 Project/validation.csv')
test = pd.read_csv('CS 583 Project/test.csv')

In [4]:
train.drop(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train = train.loc[:12000, :]

In [5]:
xtrain, xvalid, ytrain, yvalid = train_test_split(
    train['comment_text'], train['toxic'], stratify=train['toxic'], random_state=42, test_size=0.2, shuffle=True
)

In [6]:
from transformers import AutoTokenizer, AutoModel, AdamW

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\wheel\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wheel\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
class ToxicCommentsDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len=150):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        target = self.targets.iloc[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "target": torch.tensor(target, dtype=torch.float),
        }


In [8]:
train_dataset = ToxicCommentsDataset(xtrain, ytrain, tokenizer)
valid_dataset = ToxicCommentsDataset(xvalid, yvalid, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [9]:
class ToxicCommentClassifier(nn.Module):
    def __init__(self, pretrained_model_name, hidden_size=768):
        super(ToxicCommentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        output = self.dropout(pooled_output)
        return self.fc(output)


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=3):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            targets = batch["target"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs.squeeze(-1), targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {total_loss/len(train_loader)}")

        model.eval()
        valid_preds = []
        valid_targets = []
        with torch.no_grad():
            for batch in valid_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                targets = batch["target"].to(device)

                outputs = model(input_ids, attention_mask)
                valid_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                valid_targets.extend(targets.cpu().numpy())

        valid_preds_binary = [1 if p >= 0.5 else 0 for p in valid_preds]

        auc = roc_auc_score(valid_targets, valid_preds)
        acc = accuracy_score(valid_targets, valid_preds_binary)
        precision = precision_score(valid_targets, valid_preds_binary)
        recall = recall_score(valid_targets, valid_preds_binary)
        f1 = f1_score(valid_targets, valid_preds_binary)
        cm = confusion_matrix(valid_targets, valid_preds_binary)

        print(f"Validation AUC: {auc}")
        print(f"Validation Accuracy: {acc}")
        print(f"Validation Precision: {precision}")
        print(f"Validation Recall: {recall}")
        print(f"Validation F1 Score: {f1}")
        print(f"Confusion Matrix:\n{cm}")


In [11]:
model = ToxicCommentClassifier("bert-base-uncased")
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

train_model(model, train_loader, valid_loader, criterion, optimizer)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

C:\Users\wheel\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Training Loss: 0.15922690741717815
Validation AUC: 0.9804254525854208
Validation Accuracy: 0.963765097875885
Validation Precision: 0.7941176470588235
Validation Recall: 0.8325991189427313
Validation F1 Score: 0.8129032258064516
Confusion Matrix:
[[2125   49]
 [  38  189]]
Epoch 2/3, Training Loss: 0.07274720245040953
Validation AUC: 0.9838905122209208
Validation Accuracy: 0.9604331528529779
Validation Precision: 0.7481203007518797
Validation Recall: 0.8766519823788547
Validation F1 Score: 0.8073022312373225
Confusion Matrix:
[[2107   67]
 [  28  199]]
Epoch 3/3, Training Loss: 0.030884130647949253
Validation AUC: 0.9797405460609769
Validation Accuracy: 0.9625156184922948
Validation Precision: 0.8277511961722488
Validation Recall: 0.762114537444934
Validation F1 Score: 0.7935779816513762
Confusion Matrix:
[[2138   36]
 [  54  173]]


In [12]:
# Function to preprocess and predict toxic categories
def predict_toxicity(comment):
    # Tokenize and pad the input comment
    sequence = tokenizer.texts_to_sequences([comment])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    
    # Predict the toxic categories
    prediction = model.predict(padded_sequence)
    
    # Convert predictions to binary (threshold = 0.5)
    prediction_binary = (prediction > 0.5).astype(int)
    
    # Display results
    categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    for category, pred in zip(categories, prediction_binary[0]):
        print(f"{category}: {'Yes' if pred == 1 else 'No'}")

# Example comment to test
test_comment = "You're such a horrible person!"
predict_toxicity(test_comment)
test_comment1 = "You should go to hell!"
predict_toxicity(test_comment1)

AttributeError: 'BertTokenizerFast' object has no attribute 'texts_to_sequences'